In this notebook I will be using filtration methods to identify useless features.

First, load the data

In [1]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('train.csv')
other = pd.read_csv('unique_m.csv').drop(['critical_temp','material'],axis=1)
df = pd.concat([df1,other],axis=1)
original_columns = len(df.columns)
print(df.shape)
df.head()

(21263, 168)


number_of_elements  mean_atomic_mass  wtd_mean_atomic_mass  \
0                   4         88.944468             57.862692   
1                   5         92.729214             58.518416   
2                   4         88.944468             57.885242   
3                   4         88.944468             57.873967   
4                   4         88.944468             57.840143   

   gmean_atomic_mass  wtd_gmean_atomic_mass  entropy_atomic_mass  \
0          66.361592              36.116612             1.181795   
1          73.132787              36.396602             1.449309   
2          66.361592              36.122509             1.181795   
3          66.361592              36.119560             1.181795   
4          66.361592              36.110716             1.181795   

   wtd_entropy_atomic_mass  range_atomic_mass  wtd_range_atomic_mass  \
0                 1.062396          122.90607              31.794921   
1                 1.057755          122.90607              36.161939   
2                 0.975980          122.90607              35.741099   
3                 1.022291          122.90607              33.768010   
4                 1.129224          122.90607              27.848743   

   std_atomic_mass  ...   Ir   Pt   Au   Hg   Tl   Pb   Bi  Po  At  Rn  
0        51.968828  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0   0   0  
1        47.094633  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0   0   0  
2        51.968828  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0   0   0  
3        51.968828  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0   0   0  
4        51.968828  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0   0   0  

[5 rows x 168 columns]

In [2]:
cols = df.columns
# Create a new DataFrame with just the markdown
# strings
df2 = pd.DataFrame([['---',]*len(cols)], columns=cols)
#Create a new concatenated DataFrame
df3 = pd.concat([df2, df.head()])
#Save as markdown
df3.to_csv("/home/ml/projects/Superconductors_Regression/raw_data.md", sep="|", index=False)

Now remove all low variance columns - ones for which 95% of the time they have the same value.

In [3]:

# remove all columns that have a very small variance
threshold=0.05 #this gets rid of features which are the same value 95% of the time
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=threshold).fit(df)
df = pd.DataFrame(selector.transform(df),columns=df.columns[selector.get_support()])
df.shape

(21263, 132)

Now drop any duplicated features, if they exist.

In [4]:
#drop any duplicate features
df = df.transpose().drop_duplicates(keep='first').transpose()
print(df.shape)

(21263, 132)


Now drop any mutually correlated features.

In [5]:
#Now drop any correlated features
correlation_matrix = df.corr() 
mutually_correlated_features = set()  
for i in range(len(correlation_matrix.columns)):  
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            if colname!='critical_temp':
                mutually_correlated_features.add(colname)
print(len(mutually_correlated_features))
df.drop(mutually_correlated_features,axis=1,inplace=True)
df.shape

55


(21263, 77)

Now get rid of any features that have a very low correlation (abs<0.1) with the target variable, `critical_temp`.

In [6]:
#This gets rid of features that have a absolute correlation with the target less than 0.1

correlation_threshhold = 0.1

corr = pd.DataFrame(df.corr()['critical_temp'])
corr['abs'] = np.abs(corr['critical_temp'])
corr = corr.sort_values(by='abs',ascending=False).drop('abs',axis=1).dropna().reset_index()
corr = corr.rename(columns={'index':'feature','critical_temp':'corr'}).loc[1:]

low_correlated_features = list(corr[np.abs(corr['corr'])<=correlation_threshhold]['feature'])
df.drop(low_correlated_features,axis=1,inplace=True)
df.shape

(21263, 36)

In [7]:
print(str(original_columns-df.shape[1])+' features were found to be irrelevant')

132 features were found to be irrelevant


Now save the data set for modelling.

In [8]:
df.to_csv('/home/ml/projects/Superconductors_Regression/data_filtered.csv')

In [9]:
df.shape

(21263, 36)

In [10]:
cols = df.columns
# Create a new DataFrame with just the markdown
# strings
df2 = pd.DataFrame([['---',]*len(cols)], columns=cols)
#Create a new concatenated DataFrame
df3 = pd.concat([df2, df.head()])
#Save as markdown
df3.to_csv("/home/ml/projects/Superconductors_Regression/cleaned_data.md", sep="|", index=False)

# Sources:

https://stackabuse.com/applying-filter-methods-in-python-for-feature-selection/

https://stackabuse.com/applying-wrapper-methods-in-python-for-feature-selection/